In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Imports

In [3]:
import os
import warnings
import pickle
import pysam
import numpy as np
from dragonn.interpret import * 
from dragonn.vis import *
from dragonn.utils import *
from keras.models import load_model
from concise.metrics import tpr, tnr, fpr, fnr, precision, f1
from keras.models import load_model
from kerasAC.metrics import recall, specificity, fpr, fnr, precision, f1
from kerasAC.custom_losses import ambig_binary_crossentropy, ambig_mean_squared_error

Using TensorFlow backend.


2019-11-12 08:57:00,375 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



## Get Model (Regression)

In [4]:
model_prefix="/srv/scratch/annashch/bias_correction/"
reg_uncorrected=model_prefix+"uncorrected/regression/K562/DNASE.K562.regressionlabels.7"
reg_gc_1kb=model_prefix+"gc_covariate/regression/K562/DNASE.K562.regressionlabels.withgc.7"
reg_gc_200bp=model_prefix+"gc_covariate_200bp_ave/regression/K562/DNASE.K562.regressionlabels.withgc.7"
reg_gc_110bp_window=model_prefix+"gc_covariate_110bpwindow/regression/K562/DNASE.K562.regressionlabels.withgc.7"
class_uncorrected=model_prefix+"uncorrected/classification/K562/DNASE.K562.classificationlabels.7"
class_gc_1kb=model_prefix+"gc_covariate/classification/K562/DNASE.K562.classificationlabels.withgc.7"
class_gc_200bp=model_prefix+"gc_covariate_200bp_ave/classification/K562/DNASE.K562.classificationlabels.withgc.7"
class_gc_110bp_window=model_prefix+"gc_covariate_110bpwindow/classification/K562/DNASE.K562.classificationlabels.withgc.7"


##load the model 
custom_objects={"recall":recall,
                "sensitivity":recall,
                "specificity":specificity,
                "fpr":fpr,
                "fnr":fnr,
                "precision":precision,
                "f1":f1,
                "ambig_binary_crossentropy":ambig_binary_crossentropy,
                "ambig_mean_squared_error":ambig_mean_squared_error}
model_r_uncorrected=load_model(reg_uncorrected,custom_objects=custom_objects)
model_c_uncorrected=load_model(class_uncorrected,custom_objects=custom_objects)

model_r_gc_1kb=load_model(reg_gc_1kb,custom_objects=custom_objects)
model_c_gc_1kb=load_model(class_gc_1kb,custom_objects=custom_objects)

model_r_200bp=load_model(reg_gc_200bp,custom_objects=custom_objects)
model_c_200bp=load_model(class_gc_200bp,custom_objects=custom_objects)

model_r_110win=load_model(reg_gc_110bp_window,custom_objects=custom_objects)
model_c_110win=load_model(class_gc_110bp_window,custom_objects=custom_objects)


2019-11-12 08:57:12,231 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2019-11-12 08:57:12,241 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2019-11-12 08:57:12,272 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2019-11-12 08:57:12,273 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



2019-11-12 08:57:12,274 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



...

2019-11-12 08:57:16,923 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-11-12 08:57:17,187 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-11-12 08:57:18,042 [WARNING] From /users/annashch/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Get one-hot seq 

In [5]:
import pandas as pd
def get_bed_centered_at_summit(fname,flank):
    data=pd.read_csv(fname,header=None,sep='\t')
    data['chrom']=data[0]
    data['center']=data[1]+data[9]
    data['start']=data['center']-flank
    data['end']=data['center']+flank
    subset=data[['chrom','start','end']]
    inputs=[]
    for index,row in subset.iterrows(): 
        inputs.append((row['chrom'],row['start'],row['end']))
    return inputs

In [6]:
import pysam
def get_seq_from_bed(regions,ref): 
    ref=pysam.FastaFile(ref)
    seqs=[] 
    for region in regions: 
        seqs.append(ref.fetch(region[0],region[1],region[2]))        
    return seqs

In [7]:
flank=500
regions=get_bed_centered_at_summit("caprin_dnase_intersection.hg38.bed",flank)

2019-11-12 08:57:59,829 [INFO] Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2019-11-12 08:57:59,830 [INFO] NumExpr defaulting to 8 threads.


In [8]:
ref_fasta="/users/annashch/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"
seqs=get_seq_from_bed(regions,ref_fasta)

In [9]:
from dragonn.utils import one_hot_encode
seqs_onehot=one_hot_encode(seqs)

In [10]:
seqs_onehot.shape

(186, 1, 1000, 4)

## Get GC content 

In [11]:
import tiledb
#from tiledb 
gc_array="/mnt/data/annotations/gc_content_tracks/tiledb/gc_hg38_110bp_flank.chr11"
gc_array=tiledb.DenseArray(gc_array,'r')[:]['bigwig_track']
gc_110bp=[] 
for entry in regions: 
    start_pos=entry[1] 
    end_pos=entry[2] 
    gc_content=gc_array[start_pos:end_pos][400:600].mean() 
    gc_110bp.append(gc_content)
gc_110bp=np.expand_dims(np.asarray(gc_110bp),axis=1)

In [12]:
def get_gc_content(seq):
    seq=seq.upper() 
    return (seq.count('G')+seq.count('C'))/len(seq)

In [15]:
gc_1kb=[get_gc_content(seq) for seq in seqs]
gc_200bp=[get_gc_content(seq[400:600]) for seq in seqs]

gc_1kb=np.expand_dims(np.asarray(gc_1kb),axis=1)
gc_200bp=np.expand_dims(np.asarray(gc_200bp),axis=1)

## Get the gradxinput functions 

In [18]:
from keras import backend as K
def get_grad_func(cur_model,target_layer_idx): 
    return K.function(cur_model.inputs,K.gradients(cur_model.layers[target_layer_idx].output,cur_model.inputs))



In [19]:
grad_func_model_r_uncorrected=get_grad_func(model_r_uncorrected,-1)
grad_func_model_c_uncorrected=get_grad_func(model_c_uncorrected,-2)
grad_func_model_r_gc_1kb=get_grad_func(model_r_gc_1kb,-1)
grad_func_model_c_gc_1kb=get_grad_func(model_c_gc_1kb,-2)
grad_func_model_r_gc_200bp=get_grad_func(model_r_200bp,-1)
grad_func_model_c_gc_200bp=get_grad_func(model_c_200bp,-2)
grad_func_model_r_gc_110win=get_grad_func(model_r_110win,-1)
grad_func_model_c_gc_110win=get_grad_func(model_c_110win,-2)



In [20]:
grads_r_uncorrected=grad_func_model_r_uncorrected([seqs_onehot])[0]
grads_c_uncorrected=grad_func_model_c_uncorrected([seqs_onehot])[0]
grads_r_1kb=grad_func_model_r_gc_1kb([seqs_onehot,gc_1kb])[0]
grads_c_1kb=grad_func_model_c_gc_1kb([seqs_onehot,gc_1kb])[0]
grads_r_200bp=grad_func_model_r_gc_200bp([seqs_onehot,gc_200bp])[0]
grads_c_200bp=grad_func_model_c_gc_200bp([seqs_onehot,gc_200bp])[0]
grads_r_110win=grad_func_model_r_gc_110win([seqs_onehot,gc_110bp])[0]
grads_c_110win=grad_func_model_c_gc_110win([seqs_onehot,gc_110bp])[0]

In [32]:
all_grads=[grads_r_uncorrected,           
           grads_r_1kb,
           grads_r_200bp,
           grads_r_110win,
           grads_c_uncorrected,
           grads_c_1kb,
           grads_c_200bp,
           grads_c_110win]

In [33]:
subtitles=['reg. uncorrected',
          'reg. 1kb mean gc',
          'reg. 200bp mean gc',
          'reg. 110 bp smoothed',
          'class. uncorrected',
          'class. 1kb mean gc',
          'class. 200bp mean gc',
          'class. 110 bp smoothed']

In [36]:
def plot_seq_importance(subtitles, grads, x, xlim=None, ylim=None, figsize=(25, 16),title="",snp_pos=0,axes=None):
    """Plot  sequence importance score                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                                                       
    Args:                                                                                                                                                                                                                                                              
      grads: either deeplift or gradientxinput score matrix                                                                                                                                                                                                            
      x: one-hot encoded DNA sequence                                                                                                                                                                                                                                  
      xlim: restrict the plotted xrange                                                                                                                                                                                                                                
      figsize: matplotlib figure size                                                                                                                                                                                                                                  
    """
    f,axes=plt.subplots(len(grads),dpi=80,figsize=figsize)
    grads=[i.squeeze() for i in grads]
    x=x.squeeze()
    
    seq_len = x.shape[0]
    vals_to_plot=[i*x for i in grads]
    
    if xlim is None:
        xlim = (0, seq_len)
        
   
    for i in range(len(grads)):
        axes[i]=plot_bases_on_ax(vals_to_plot[i],axes[i],show_ticks=True)
        axes[i].set_xlim(xlim)
        axes[i].set_title(subtitles[i],fontsize=12)
        
    plt.xticks(list(range(xlim[0], xlim[1], 5)))
    plt.subplots_adjust(hspace=0.4)
    plt.savefig(title+'.png',dpi=80)

In [ ]:
for i in range(len(regions)):
    region=regions[i]
    cur_title='DNAse:'+':'.join([str(j) for j in region])
    cur_grads=[a[i] for a in all_grads]
    plot_seq_importance(subtitles, 
                        cur_grads, 
                        seqs_onehot[i],
                        title=cur_title,
                        xlim=(400,600))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
